In [1]:
import requests
import csv
import time
import re
from bs4 import BeautifulSoup

In [4]:
API_URL = "https://api.nhathuoclongchau.com.vn/lccus/prod-cms/api/v2/categories/category-topic/articles"
INPUT_CATEGORY_LINKS = '../data/raw/category_links.txt'
OUTPUT_ALL_CATEGORY_LINKS = '../data/raw/articles_links.csv'

In [5]:
urls = []

try:
    with open(INPUT_CATEGORY_LINKS, 'r', encoding='utf-8') as f:
        for line in f:
            url = line.strip()
            if url:
                urls.append(url)
    print(f"Đã tải {len(urls)} URL từ tệp: {INPUT_CATEGORY_LINKS}")
    
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp {INPUT_CATEGORY_LINKS}. Vui lòng kiểm tra lại đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc tệp: {e}")

Đã tải 18 URL từ tệp: ../data/raw/category_links.txt


In [6]:
def fetch_articles(slug, default_category_name, limit=16):
    offset = 0
    all_articles = []

    print(f"\n🚀 Bắt đầu lấy bài viết cho danh mục: {default_category_name} (slug: {slug})")

    while True:
        params = {"slug": slug, "limit": limit, "offset": offset}
        headers = {"User-Agent": "Mozilla/5.0"}

        res = requests.get(API_URL, params=params, headers=headers, timeout=10)
        res.raise_for_status()
        data = res.json()

        inner_data = data.get("data", {})
        articles = inner_data.get("articles", [])
        total = inner_data.get("totalCount", 0)

        if not articles:
            break

        for a in articles:
            if isinstance(a, dict):
                all_articles.append({
                    "category": default_category_name,
                    "disease": a.get("name"),
                    "link": f"https://nhathuoclongchau.com.vn/{a.get('slug')}"
                })
            else:
                print(f"Warning: Expected a dictionary, but got type {type(a)}: {a}")

        offset += limit
        print(f"✅ Lấy {len([item for item in articles if isinstance(item, dict)])} bài (tổng {len(all_articles)}/{total}) trong '{default_category_name}'")

        if len(all_articles) >= total:
            break

        time.sleep(0.5)

    return all_articles

In [7]:
def fetch_all_nhom_benh_category_slugs():
    if not urls:
        print("The 'urls' variable is empty or not found. Cannot determine category slugs.")
        return []

    category_slugs = set()
    for full_url in urls:
        match = re.search(r'nhathuoclongchau\.com\.vn/(.*)', full_url)
        if match:
            slug = match.group(1)
            if slug.startswith('benh/nhom-benh/') and slug != 'benh/nhom-benh/':
                category_slugs.add(slug)

    if not category_slugs:
        print("No specific 'nhom-benh' category slugs could be extracted from the 'urls' variable.")

    return sorted(list(category_slugs))

In [8]:
category_slugs_to_crawl = fetch_all_nhom_benh_category_slugs()

if not category_slugs_to_crawl:
    print("Không tìm thấy danh mục 'nhom-benh' nào để crawl.")
else:
    print(f"Tìm thấy {len(category_slugs_to_crawl)} danh mục 'nhom-benh':")
    for s in category_slugs_to_crawl:
        print(f"- {s}")

    all_articles_from_all_categories = []

    for slug in category_slugs_to_crawl:

        category_name_for_output = slug.split('/')[-1]

        articles_for_category = fetch_articles(slug, category_name_for_output)
        all_articles_from_all_categories.extend(articles_for_category)

    if all_articles_from_all_categories:
        with open(OUTPUT_ALL_CATEGORY_LINKS, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["category", "disease", "link"])
            writer.writeheader()
            writer.writerows(all_articles_from_all_categories)

        print(f"\n🎉 Hoàn tất! Đã lưu {len(all_articles_from_all_categories)} bài từ tất cả danh mục vào: {OUTPUT_ALL_CATEGORY_LINKS}")
    else:
        print("Không có bài viết nào được tìm thấy để lưu.")

Tìm thấy 18 danh mục 'nhom-benh':
- benh/nhom-benh/co-xuong-khop
- benh/nhom-benh/da-toc-mong
- benh/nhom-benh/di-ung
- benh/nhom-benh/ho-hap
- benh/nhom-benh/mat
- benh/nhom-benh/mau
- benh/nhom-benh/noi-tiet-chuyen-hoa
- benh/nhom-benh/rang-ham-mat
- benh/nhom-benh/suc-khoe-gioi-tinh
- benh/nhom-benh/suc-khoe-sinh-san
- benh/nhom-benh/tai-mui-hong
- benh/nhom-benh/tam-than
- benh/nhom-benh/than-kinh-tinh-than
- benh/nhom-benh/than-tiet-nieu
- benh/nhom-benh/tieu-hoa
- benh/nhom-benh/tim-mach
- benh/nhom-benh/truyen-nhiem
- benh/nhom-benh/ung-thu

🚀 Bắt đầu lấy bài viết cho danh mục: co-xuong-khop (slug: benh/nhom-benh/co-xuong-khop)
✅ Lấy 16 bài (tổng 16/177) trong 'co-xuong-khop'
✅ Lấy 16 bài (tổng 32/177) trong 'co-xuong-khop'
✅ Lấy 16 bài (tổng 48/177) trong 'co-xuong-khop'
✅ Lấy 16 bài (tổng 64/177) trong 'co-xuong-khop'
✅ Lấy 16 bài (tổng 80/177) trong 'co-xuong-khop'
✅ Lấy 16 bài (tổng 96/177) trong 'co-xuong-khop'
✅ Lấy 16 bài (tổng 112/177) trong 'co-xuong-khop'
✅ Lấy 16 bài 